# Google Colab initialization

This section will help you interface with Google Drive and clone the git repository where the code lives. These steps **aren't necessary if you are running locally**. First, make sure you have opened the notebook in Google Colab (use the below button if ncessary) and logged into your Google account.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/tanderson11/covid_households/blob/main/notebooks/ViolinsAndPowerCalc.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%mkdir /content/gdrive/My\ Drive/github/
%cd /content/gdrive/My\ Drive/github/
# Thayer has his files located here instead
#%cd /content/gdrive/My\ Drive/github/paper_push


In [ ]:
# If you've forked the repository, point to your own username and repository name (if different)
repo_owner="tanderson11"
repository="covid_households"

!git config --global user.email "tanderson11@gmail.com"
!git config --global user.name "Thayer Anderson"

In [ ]:
!git clone https://github.com/tanderson11/covid_households.git

In [ ]:
%cd covid_households/
!ls -a

In [ ]:
!git checkout main
!git pull

In [ ]:
%cd ./notebooks

# Module initialization

In [8]:
%cd ../covid_households
import recipes
import interventions
import traits

/Users/thayer/covid_households/covid_households


# Vaccine trial

This cell configures the parameters that are shared between the vaccinated arm and the control arm.

The traits of susceptibility and infectivity refer to population variation in these traits of individuals (i.e. it's unrelated to the vaccine). You can set these to a `ConstantTrait` for no variation or to a `LognormalTrait` to achieve variation in the population.

In [9]:
susceptibility = traits.ConstantTrait()
#susceptibility = traits.LognormalTrait.from_natural_mean_variance(mean=1.0, variance=0.5)
infectivity = traits.ConstantTrait()
#infectivity = traits.LognormalTrait.from_natural_mean_variance(mean=1.0, variance=2.0)

# {size: # of households of that size}
sizes = {5:100, 6:100}
trials = 10
household_beta = 0.05

This cell configures the vaccine.

The `shape` refers to how the vaccine is applied to the population. `InterveneOnFirst` means the "first" individual in each household is vaccinated (since household order is totally random, this effectively vaccinates a random individual in each household, but be careful not to break the symmetry of individuals by introducing something *else* that cares about particular individuals).

The `vaccine` determines the (relative per-contact) susceptibility and infectivity of vaccinated individuals. For example, `sus_factor=0.2` says that a vaccinated person is only $20\%$ as likely as an unvaccinated person to be infected per contact.

In [10]:
shape = interventions.InterveneOnFirst()
vaccine = interventions.ConstantFactorIntervention(shape, sus_factor=0.2, inf_factor=0.3)

These cells simulate forwards in time. They are purely configured above, you shouldn't need to touch them except for advanced uses.

In [11]:
vax_model = recipes.Model(intervention=vaccine)
vax_df = vax_model.run_trials(household_beta=household_beta, sizes=sizes, trials=trials, sus=susceptibility, inf=infectivity, as_counts=False)

vax_df.groupby('trial').sum()

,size,infections,intervention and infection,total interventions
trial,,,,
0,1100,656,56,200.0
1,1100,614,56,200.0
2,1100,617,56,200.0
3,1100,586,55,200.0
4,1100,655,72,200.0
5,1100,619,64,200.0
6,1100,654,70,200.0
7,1100,588,47,200.0
8,1100,661,78,200.0


In [12]:
# 1.0 and 1.0 because the placebo has no effect
control_model = recipes.Model(intervention=interventions.ConstantFactorIntervention(shape, 1.0, 1.0))
control_df = control_model.run_trials(household_beta=household_beta, sizes=sizes, trials=trials, sus=susceptibility, inf=infectivity, as_counts=False)

control_df.groupby('trial').sum()

,size,infections,intervention and infection,total interventions
trial,,,,
0,1100,769,147,200.0
1,1100,745,132,200.0
2,1100,749,135,200.0
3,1100,797,139,200.0
4,1100,771,133,200.0
5,1100,798,143,200.0
6,1100,832,154,200.0
7,1100,668,120,200.0
8,1100,734,134,200.0


# Quantifying vaccine effects

Defining and calculating $\text{VE}_{\text{S}}$:

$$ \text{VE}_{\text{S}} = \left.1 - \frac{AR_v}{AR_p} = 1 - {n_v^+\ /\ (n_v^+ + n_v^-)} \middle/ \right. {n_p^+\ /\ (n_p^+ + n_{p}^-)}{}$$

Defining and calculating $\text{VE}_{\text{contacts}}$:

$$ \text{VE}_{\text{contact}} = \left.1 - \frac{AR_u}{AR_{np}} = 1 - {\ n_u^+/\ (n_u^+ + n_u^-)} \middle/ \right. {n_{np}^+\ /\ (n_{np}^+ + n_{np}^-)}$$

where $\pm$ refers to infected and uninfected individuals, $n_v$ refers to vaccinated individuals (who received the real vaccine), $n_u$ refers to unvaccinated individuals (in households where the vaccine was administered), $n_p$ refers to placebo-receiving individuals, and $n_{np}$ refers to individuals who received no placebo (in households where the placebo was administered).

The term $AR$ (attack rate) is used as defined in the literature and should not be understood as a rate, but simply as the observed frequency of infections among a group such that $AR_v$, for example, is defined as the fraction of vaccinated individuals who were in fact infected.

Defining and calculating $\text{VE}_{\text{total}}$:

$$ \text{VE}_{\text{total}} = \left.1 - \frac{AR_{HV}}{AR_{HP}} = 1 - {n_{HV}^+\ /\ (n_{HV}^+ + n_{HV}^-)} \middle/ \right. {n_{HP}^+\ /\ (n_{HP}^+ + n_{HP}^-)}{}$$

Heree the notation has changed slightly to allow $n_{HV}$ to refer to the total number of individuals in households that received a vaccine, ie $n_{HV} = n_{v} + n_{u}$.

In [13]:
import pandas as pd
import scipy

def ves(vax_df, control_df):
    """Vaccination effect on susceptibility using the placebo RR as baseline (equation 1/2 in Betz)"""
    print("Calculating VEs ...\n")

    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["intervention and infection"].sum() / vg["total interventions"].sum()

    cg = control_df.groupby(["trial"])
    cgs = cg.sum()
    f_c = cg["intervention and infection"].sum() / cg["total interventions"].sum()     

    # fisher exact test record actual number of events: columns either vaccinated or in household with vaccination vs other and rows = individual was infected vs not

    # fisher exact test : comparing primary participants in households

    ##             placebo | vaccinated
    ##  uninfected
    ##  -----
    ##  infected

    fisher_df = pd.concat([cgs["total interventions"] - cgs["intervention and infection"], vgs["total interventions"] - vgs["intervention and infection"], cgs["intervention and infection"], vgs["intervention and infection"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    ve = 1. - f_v / f_c
    ve.name = "VE"

    return pd.concat([ve, p], axis=1)

def vecontact(vax_df, control_df):
    print("Calculating VEcontact ...\n")
    vax_df = vax_df.copy()
    vax_df["total unvaccinated"] = vax_df["size"] - vax_df["total interventions"]
    vax_df["unvaccinated and infected"] = vax_df["infections"] - vax_df["intervention and infection"]
    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["unvaccinated and infected"].sum() / vg["total unvaccinated"].sum()

    control_df = control_df.copy()
    cg = control_df.groupby(["trial"])
    control_df["total unvaccinated"] = control_df["size"] - control_df["total interventions"]
    control_df["unvaccinated and infected"] = control_df["infections"] - control_df["intervention and infection"]
    cgs = cg.sum()
    f_c = cg["unvaccinated and infected"].sum() / cg["total unvaccinated"].sum()
    
    ve = 1. - f_v / f_c
    #ve.name = "VEcontact"
    ve.name = "VE"

    # fisher exact test : comparing households by type but only unvaccinated
    ##             control hh secondary (no placebo) | vaccinated hh secondary (no vax)
    ##  uninfected
    ##  -----
    ##  infected


    fisher_df = pd.concat([cgs["total unvaccinated"]-cgs["unvaccinated and infected"], vgs["total unvaccinated"]-vgs["unvaccinated and infected"], cgs["unvaccinated and infected"], vgs["unvaccinated and infected"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    return pd.concat([ve, p], axis=1)

def vetotal(vax_df, control_df):
    print("Calculating VEtotal ...\n")
    vg = vax_df.groupby(["trial"])
    vgs = vg.sum()
    f_v = vg["infections"].sum() / vg["size"].sum()

    cg = control_df.groupby(["trial"])
    cgs = cg.sum()
    f_c = cg["infections"].sum() / cg["size"].sum()

    ve = 1. - (f_v)/(f_c)
    #ve.name = "VEtotal"
    ve.name = "VE"

    # fisher exact test : comparing households by type

            ##             control hh | vaccinated hh
    ##  uninfected
    ##  -----
    ##  infected


    fisher_df = pd.concat([cgs["size"]-cgs["infections"], vgs["size"]-vgs["infections"], cgs["infections"], vgs["infections"]], axis=1)
    fisher_df.columns =["cuinfected", "vuinfected", "cinfected", "vinfected"]
    p = fisher_df.apply(lambda row: (scipy.stats.fisher_exact([[row["cuinfected"], row["vuinfected"]], [row["cinfected"], row["vinfected"]]]))[1], axis=1) # index 1 to get p value
    p.name = "fisher p value"

    return pd.concat([ve, p], axis=1)

In [14]:
ves(vax_df, control_df)

Calculating VEs ...



,VE,fisher p value
trial,,
0,0.619048,5.026997e-20
1,0.575758,2.814829e-14
2,0.585185,2.529031e-15
3,0.604317,3.462578e-17
4,0.458647,1.473286e-09
5,0.552448,2.603986e-15
6,0.545455,1.938521e-17
7,0.608333,1.422923e-13
8,0.417910,2.937811e-08


In [17]:
vecontact(vax_df, control_df)

Calculating VEcontact ...



,VE,fisher p value
trial,,
0,0.035370,2.890847e-01
1,0.089723,7.572971e-03
2,0.086319,1.001312e-02
3,0.193009,3.208991e-10
4,0.086207,6.409241e-03
5,0.152672,6.372284e-07
6,0.138643,1.609052e-06
7,0.012774,7.723679e-01
8,0.028333,4.268867e-01


In [16]:
vetotal(vax_df, control_df)

Calculating VEtotal ...



,VE,fisher p value
trial,,
0,0.146944,5.580855e-07
1,0.175839,1.116241e-08
2,0.176235,8.140250e-09
3,0.264743,1.334555e-20
4,0.150454,2.730554e-07
5,0.224311,1.836171e-15
6,0.213942,5.918417e-16
7,0.119760,6.629873e-04
8,0.099455,1.431217e-03
